###  Simulate `build_gaussian_tf` function 
- Setup model, load weights and data generators 
- get items from data generator
- pass through network

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_dev_notebook, load_model

model, dataset_train, train_generator, config = prep_dev_notebook(init_with = 'last', FCN_layers = False)

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
load_model(model, init_with='last')

In [ ]:
model.layer_info()
# model.keras_model.outputs[0].name

In [ ]:

# model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
model_output = get_layer_output_1(model.keras_model, train_batch_x, [1,2,3,4,5,7,9,10,17,19], 1)


#### Load input and output tensors

In [ ]:
print(len(model_output))

rpn_class                 = model_output[0]          # layer:  1   shape: (5, 4092, 2)
rpn_bbox                  = model_output[1]          # layer:  2   shape: (5, 4092, 4)
rpn_proposal_rois         = model_output[2]          # layer:  3   shape: (5, 2000, 4)
output_rois               = model_output[3]          # layer:  4   shape: (5, 32, 4)
target_class_ids          = model_output[4]          # layer:  5   shape: (5, 32)
roi_gt_boxes              = model_output[5]          # layer:  7   shape: (5, 32, 4)
mrcnn_class               = model_output[6]          # layer:  9   shape: (5, 32, 4)
mrcnn_bbox                = model_output[7]          # layer: 10   shape: (5, 32, 4, 4)
pred_heatmap_scores       = model_output[8]          # layer: 17   shape: (5, 4, 32, 8)
pred_tensor               = model_output[9]          # layer: 19   shape: (5, 4, 32, 6)
# rpn_class_logits   = model_output[0]
# rpn_class          = model_output[0]
# rpn_bbox           = model_output[1]
# rpn_proposal_rois  = model_output[2]
# output_rois        = model_output[3]
# target_class_ids   = model_output[5]
# target_bbox_deltas = model_output[6]

# print(type(model_output[4]))
# print(type(output_rois))
for i in model_output:
    print( i.shape)

In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
# input_rpn_match  =  train_batch_x[2]
# input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
# input_gt_masks     = train_batch_x[6]
print(input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
print('height/width', h,w)
input_gt_bboxes_norm = input_gt_bboxes / [h,w,h,w]

# print(' input_rpn_match    ', input_rpn_match.shape)
# print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids   ', input_gt_class_ids.shape)
print(input_gt_class_ids)
print(' input_gt_bboxes      ', input_gt_bboxes.shape)
print(input_gt_bboxes)  
print(' input_gt_bboxes_norm ', input_gt_bboxes_norm.shape)
print( input_gt_bboxes_norm)

## Proposal Layer

In [ ]:
import mrcnn.utils as utils
sess = KB.get_session()


In [ ]:
def apply_box_deltas_graph(boxes, deltas):
    """
    Applies the given deltas to the given boxes.

    x,y,w,h : Bounding Box coordinates, width, and height

    Boxes:  is the (y1, x1, y2, x2) of the anchor boxes
    deltas: [dy, dx, log(dh), log(dw)] 
            is the predicted bounding box returned from the RPN layer (in test phase)
            These are considered the targets or ground truth 
    Refer to Bounding Box Regression - R-CNN paper 
    Regression targets are calculated as follows:   
      tx = (GTx - PRx)/PRw       ty = (Gy - Py)/Ph
      th = log(Gw/Pw)         tw = log(Gh/Ph)
    ---------------------------------------------------------------------------------            
    
    boxes:  [N, 4] where each row is [y1, x1, y2, x2]
    deltas: [N, 4] where each row is [dy, dx, log(dh), log(dw)]
    """
    # Convert to y, x, h, w
    height   = boxes[:, 2] - boxes[:, 0]
    width    = boxes[:, 3] - boxes[:, 1]
    center_y = boxes[:, 0] + 0.5 * height
    center_x = boxes[:, 1] + 0.5 * width
    
    # Apply deltas
    center_y += deltas[:, 0] * height
    center_x += deltas[:, 1] * width
    height   *= tf.exp(deltas[:, 2])
    width    *= tf.exp(deltas[:, 3])
    
    # Convert back to y1, x1, y2, x2
    y1 = center_y - 0.5 * height
    x1 = center_x - 0.5 * width
    y2 = y1 + height
    x2 = x1 + width
    result = tf.stack([y1, x1, y2, x2], axis=1, name="apply_box_deltas_out")
    return result


def clip_boxes_graph(boxes, window):
    """
    clip refined anchor boxes such that they remain within the dimensions of the image 
    boxes:  [N, 4] each row is y1, x1, y2, x2
    window: [4] in the form y1, x1, y2, x2 
    """
    # Split corners
    wy1, wx1, wy2, wx2 = tf.split(window, 4)   #  0, 0 , 128,128
    y1 , x1 , y2 , x2  = tf.split(boxes, 4, axis=1)
    
    # Clip
    y1 = tf.maximum(tf.minimum(y1, wy2), wy1)   # ensure  wy1 <= y1 <= wy2
    x1 = tf.maximum(tf.minimum(x1, wx2), wx1)
    y2 = tf.maximum(tf.minimum(y2, wy2), wy1)
    x2 = tf.maximum(tf.minimum(x2, wx2), wx1)
    clipped = tf.concat([y1, x1, y2, x2], axis=1, name="clipped_boxes")
    return clipped

def suppress_small_boxes_graph(boxes, scores, area_threshold ):
    """
    supress boxes with area less than area_threshold
    boxes:  [N, 4] each row is y1, x1, y2, x2
    
    """
    bx_area = (boxes[...,2]-boxes[...,0])*(boxes[...,3]-boxes[...,1])
    selected_idxs   = tf.where(tf.greater_equal(bx_area, area_threshold))
    selected_boxes  = tf.gather_nd(boxes, selected_idxs)
    selected_scores = tf.gather_nd(scores, selected_idxs)
    padding   = tf.maximum(tf.shape(boxes)[0] - tf.shape(selected_boxes)[0], 0)

#     print(' box area       : ', bx_area.shape)    
#     print(' selected_idxs  : ', tf.shape(selected_idxs).eval())
#     print(' selected scores: ', tf.shape(selected_scores).eval())
#     print(' Req padding    : ', padding.eval())
    
    selected_boxes  = tf.pad(selected_boxes, [(0, padding), (0, 0)])    
    selected_scores = tf.pad(selected_scores, [(0, padding)])    
    return selected_boxes, selected_scores

def nms(normalized_boxes, scores):
    proposal_count = model.config.POST_NMS_ROIS_TRAINING
    rms_threshold  = model.config.RPN_NMS_THRESHOLD
    indices = tf.image.non_max_suppression(normalized_boxes, 
                                           scores, 
                                           proposal_count,
                                           nms_threshold, 
                                           name="rpn_non_max_suppression")

    proposals = tf.gather(normalized_boxes, indices)
    # Pad if needed
    padding   = tf.maximum(proposal_count - tf.shape(proposals)[0], 0)
    proposals = tf.pad(proposals, [(0, padding), (0, 0)])
    return proposals

In [ ]:
with sess.as_default():
    scores = rpn_class[:, :, 1]
    print(scores.shape)
    deltas = rpn_bbox
    print('deltas shape', deltas.shape)
    deltas = deltas * np.reshape(model.config.RPN_BBOX_STD_DEV, [1, 1, 4])
    print('deltas shape', deltas.shape)
    anchors = model.anchors
    print('model.anchors.shape', model.anchors.shape)
    pre_nms_limit = min(6000, model.anchors.shape[0])
    print('pre nms limit', pre_nms_limit)

    ix = tf.nn.top_k(scores, pre_nms_limit, sorted=True,name="top_anchors").indices

    ## gather top scores (pre_nms_limit = min(6000, # anchors) number of scores from scores)        
    scores  = utils.batch_slice([scores, ix], lambda x, y: tf.gather(x, y), model.config.IMAGES_PER_GPU) 
    print(' selected scores: ',scores.shape)

    ## get corrsponding deltas generated by RPN
    deltas  = utils.batch_slice([deltas, ix], lambda x, y: tf.gather(x, y), model.config.IMAGES_PER_GPU)
    print(' selected deltas: ',deltas.shape)
    anchors = utils.batch_slice(         ix , lambda x   : tf.gather(anchors, x), model.config.IMAGES_PER_GPU, names=["pre_nms_anchors"])
    print(' selected anchors: ',anchors.shape)
    boxes = utils.batch_slice([anchors, deltas],
                              lambda x, y: apply_box_deltas_graph(x, y),model.config.IMAGES_PER_GPU,
                              names=["refined_anchors"])
    print(' delta applied boxes :', boxes.shape)

    # Clip to image boundaries. [batch, N, (y1, x1, y2, x2)]
    height, width = model.config.IMAGE_SHAPE[:2]
    window = np.array([0, 0, height, width]).astype(np.float64)
    print(' window is ', window)
    clipped_boxes  = utils.batch_slice(boxes, 
                                   lambda x: clip_boxes_graph(x, window), model.config.IMAGES_PER_GPU,
                                   names=["refined_anchors_clipped"])

    print(' clipped boxes :', clipped_boxes.shape)
    
    ## Suppress proposal boxes (and  corresponding score) if the area is less than ROI_AREA_THRESHOLD
    roi_area_threshold = 2
    mod_boxes, mod_scores = utils.batch_slice([clipped_boxes,scores], 
                            lambda x, y: suppress_small_boxes_graph(x, y, roi_area_threshold), model.config.IMAGES_PER_GPU,
                            names=["mod_boxes", "mod_scores"])  
    print(' mod boxes :', tf.shape(mod_boxes).eval())
    print(' mod_scores:', tf.shape(mod_scores).eval())   
    
    normalized_boxes = tf.cast(mod_boxes / np.array([[height, width, height, width]]), tf.float32)
    print(' normalized boxes:', normalized_boxes.shape)

    proposals = utils.batch_slice([normalized_boxes, mod_scores], nms, model.config.IMAGES_PER_GPU)
    print('     Output: Prposals shape : ', proposals.shape, tf.shape(proposals).eval())
    

####  Analyze proposals results 

In [ ]:
sess = KB.get_session()
with sess.as_default():
    proposals = tf.identity(rpn_proposal_rois)  # <--- this uses the results from the model 

    bx_area = (proposals[...,2]-proposals[...,0])*(proposals[...,3]-proposals[...,1])
    print(' proposals :', tf.shape(proposals).eval())
    print(' box area : ', tf.shape(bx_area).eval())
    
    selected_idxs = tf.where(tf.less_equal(bx_area, (2/(128*128))) )
    print('selected bx:', tf.shape(selected_idxs).eval())
    
    print(selected_idxs.eval())
    selected_area      = tf.gather_nd(bx_area  , selected_idxs)
    selected_proposals = tf.gather_nd(proposals, selected_idxs)
    print('selected proposals shape', tf.shape(selected_proposals).eval())
    print(selected_proposals[0:30].eval())
    print('selected area shape', tf.shape(selected_area).eval())
    print(selected_area[0:30].eval())
    

####  Analyze bounding box areas 

In [ ]:
with sess.as_default():
    print(' boxes :', boxes.shape)
    for i in [0,10,17,25,26,34,39]:
        print(i, ' non-clipped ', boxes[0, i].eval())
    bx_area = (boxes[...,2]-boxes[...,0])*(boxes[...,3]-boxes[...,1])
    print(' box area : ', bx_area.shape)
    np.set_printoptions(linewidth=130,precision=4,threshold=4096)
    print(bx_area[:, :20].eval(session=sess))
    small_idxs = tf.where(bx_area < 1)
    print('small bx:', tf.shape(small_idxs).eval())
    print(small_idxs[0:10].eval())
    small_area  = tf.gather_nd(bx_area, small_idxs)
    small_boxes = tf.gather_nd(boxes, small_idxs)
    print('small boxes shape', tf.shape(small_boxes).eval())
    print(small_boxes[0:30].eval())
    print('small area shape', tf.shape(small_area).eval())
    print(small_area[0:30].eval())
    

## Detection Layer

####  Setup tensors to be passed to `detections_target_graph()`    - Detection Target Layer

In [ ]:
import mrcnn.utils  as utils
from mrcnn.detect_tgt_layer import overlaps_graph
# sess = KB.get_session()
# with  sess.as_default():
try:
    sess.close()
    print('session was deleted ')
except:
    print('Session was not defined ')
    pass
sess = tf.InteractiveSession()

image_id = 1

proposals    = KB.identity(rpn_proposal_rois)[image_id]
gt_class_ids = KB.identity(input_gt_class_ids)[image_id]
gt_boxes     = KB.cast(KB.identity(input_gt_bboxes_norm), dtype='float32')[image_id]
# gt_masks     = KB.identity(input_gt_masks)
print('rpn_roi_proposals')
print(proposals.dtype, gt_class_ids.dtype, gt_boxes.dtype)
print(proposals.shape)
print(proposals.eval())
print('gt_class_ids')
print(gt_class_ids.shape)
print(gt_class_ids.eval())
print('gt_boxes')
print(gt_boxes.shape)
print(gt_boxes.eval())
# proposals    = rpn_proposal_rois[1]
# gt_class_ids = input_gt_class_ids[1]
# gt_boxes     = input_normlzd_gt_boxes[1]
# gt_masks     = input_gt_masks[1]
# config       = model.config

### def `dev_detection_targets_graph`

In [ ]:
# def dev_detection_targets_graph(proposals, gt_class_ids, gt_boxes, gt_masks, config):
 
'''
as of 16-0-2018
Generates detection targets for one image. Subsamples proposals and
generates target class IDs, bounding box deltas, and masks for each.

Inputs:
-------
proposals:          [N, 2000, (y1, x1, y2, x2)] in normalized coordinates. 
                         Might be zero padded if there are not enough proposals.
gt_class_ids:       [MAX_GT_INSTANCES] int class IDsMx2)] in normalized coordinates.
gt_masks:           [height, width, MAX_GT_INSTANCES] of boolean type.

Returns:            Target ROIs and corresponding class IDs, bounding box shifts, and masks.
--------
rois:               [TRAIN_ROIS_PER_IMAGE, (y1, x1, y2, x2)] in normalized coordinates
class_ids:          [TRAIN_ROIS_PER_IMAGE]. Integer class IDs. Zero padded.
deltas:             [TRAIN_ROIS_PER_IMAGE, NUM_CLASSES, (dy, dx, log(dh), log(dw))]
                    Class-specific bbox refinments.
masks:              [TRAIN_ROIS_PER_IMAGE, height, width). Masks cropped to bbox
                    boundaries and resized to neural network output size.

Note: Returned arrays might be zero padded if not enough target ROIs.

''' 
# Assertions
asserts = [
    tf.Assert(tf.greater(tf.shape(proposals)[0], 0), [proposals], name="roi_assertion"),
]

with tf.control_dependencies(asserts):
    proposals = tf.identity(proposals)
# print('>>> detection_targets_graph ')
# print('     propsals.shape        :',  proposals.shape, proposals.get_shape(), KB.int_shape(proposals) )
# print('     gt_boxes.shape        :',  gt_boxes.shape ,    KB.int_shape(gt_boxes)   )
# print('     gt_class_ids.shape    :',  gt_class_ids.shape, KB.int_shape(gt_class_ids))
# print('     gt_masks.shape        :',  gt_masks.shape ,    KB.int_shape(gt_masks)   )

# Remove zero padding   
# non_zeros returns indicies to valid bboxes, which we use to index gt_class_ids, and gt_masks
proposals, non_zeros1= utils.trim_zeros_graph(proposals, name="trim_proposals")
gt_boxes, non_zeros = utils.trim_zeros_graph(gt_boxes , name="trim_gt_boxes")
gt_class_ids        = tf.boolean_mask(gt_class_ids, non_zeros, name="trim_gt_class_ids")
# gt_masks            = tf.gather(gt_masks, tf.where(non_zeros)[:, 0], axis=2,name="trim_gt_masks")

print(tf.reduce_all(non_zeros1).eval())
print(tf.shape(proposals).eval())
print(non_zeros.eval())
print(tf.shape(gt_boxes).eval())
print(gt_boxes.eval())

#------------------------------------------------------------------------------------------
# Handle COCO crowds
# A crowd box in COCO is a bounding box around several instances. Exclude
# them from training. A crowd box is given a negative class ID.
#------------------------------------------------------------------------------------------
# tf.where : returns the coordinates of true elements of  the specified conditon.
#            The coordinates are returned in a 2-D tensor where the first dimension (rows) 
#            represents the number of true elements, and the second dimension (columns) 
#            represents the coordinates of the true elements. 
#            Keep in mind, the shape of the output tensor can vary depending on how many 
#            true values there are in input. Indices are output in row-major order.
#
# tf.gather: Gather slices from params axis (default = 0) according to indices.
#            indices must be an integer tensor of any dimension (usually 0-D or 1-D). 
#            Produces an output tensor with shape:
#                   params.shape[:axis] + indices.shape + params.shape[axis + 1:] 
#
# tf.squeeze: Removes dimensions of size 1 from the shape of a tensor.
#            Given a tensor input, this operation returns a tensor of the same type with 
#            all dimensions of size 1 removed. If you don't want to remove all size 1 
#            dimensions, you can remove specific size 1 dimensions by specifying axis.
#------------------------------------------------------------------------------------------

crowd_ix        = tf.where(gt_class_ids < 0)[:, 0]
non_crowd_ix    = tf.where(gt_class_ids > 0)[:, 0]
crowd_boxes     = tf.gather(gt_boxes, crowd_ix)
# crowd_masks     = tf.gather(gt_masks, crowd_ix, axis=2)
gt_class_ids    = tf.gather(gt_class_ids, non_crowd_ix)
gt_boxes        = tf.gather(gt_boxes, non_crowd_ix)
# gt_masks        = tf.gather(gt_masks, non_crowd_ix, axis=2)
print(non_crowd_ix.eval())
print(gt_boxes.eval())

# Compute overlaps with crowd boxes [anchors, crowds]
crowd_overlaps  = overlaps_graph(proposals, crowd_boxes)
crowd_iou_max   = tf.reduce_max(crowd_overlaps, axis=1)
no_crowd_bool   = (crowd_iou_max < 0.001)

# Compute overlaps matrix [proposals, gt_boxes] - The IoU between 
# proposals and gt_boxes (non-crowd gt boxes, designated by classId < 0 in Coco)
# overlaps is 
# compute max of elements across axis 1 of overlaps tensor. 

In [ ]:
np.set_printoptions(linewidth=150, precision=6, suppress = True)
overlaps        = overlaps_graph(proposals, gt_boxes)
print(tf.shape(overlaps).eval())
print(overlaps[0:500].eval())

In [ ]:
roi_iou_max     = tf.reduce_max(overlaps, axis=1)
print('     overlaps.shape        :',  tf.shape(overlaps).eval())
print(overlaps.eval())
zero_idxs = tf.where(tf.greater_equal(overlaps,0.5))
print(tf.shape(zero_idxs).eval())

## 1. Determine indices of postive ROI propsal boxes
#    Identify ROI proposal boxes that have an IoU >= 05 overlap with some gt_box, and store 
#    indices into positive_indices
positive_roi_bool     = (roi_iou_max >= 0.5)
positive_indices      = tf.where(positive_roi_bool)[:,0]
print(tf.shape(positive_roi_bool).eval())
print(tf.shape(positive_indices).eval())
print(positive_indices.eval())



## 2. Determine indices of negative ROI proposal boxes
#    those with < 0.5 with every GT box and are not crowds bboxes 
# the where creates a array with shape [# of answers, 1] so we use [:, 0] after
## current method
negative_indices      = tf.where(tf.logical_and(roi_iou_max < 0.5, no_crowd_bool))[:, 0]

## new method
# this modification will determine negative ROI proposal boxes but in addition, 
# will suppress the zero RoIs from the indicies 
# note that   ( negative_bool         = ~positive_roi_bool)
# negative_nonzero_bool = tf.logical_and(~positive_roi_bool, (roi_iou_max > 0))
# negative_nonzero_bool = tf.logical_and(negative_nonzero_bool, no_crowd_bool)
# negative_indices2     = tf.where(negative_nonzero_bool) [:, 0]

## 3. Subsample positive ROIs based on ROI_POSITIVE_RATIO
#    Aim for 33% positive (config.ROI_POSITIVE_RATIO = 0.33)
#    Positive ROIs   33% of config.TRAIN_ROIS_PER_IMAGE ~  11
positive_count        = int(config.TRAIN_ROIS_PER_IMAGE * config.ROI_POSITIVE_RATIO)
positive_indices      = tf.random_shuffle(positive_indices)[:positive_count]
positive_count        = tf.shape(positive_indices)[0]

print(positive_indices.eval())
print(positive_count.eval())

## 4. Add Negative ROIs. Add enough to maintain positive:negative ratio
#     negative_count = int((positive_count / config.ROI_POSITIVE_RATIO) - positive_count)
r = 1.0 / config.ROI_POSITIVE_RATIO
negative_count        = tf.cast(r * tf.cast(positive_count, tf.float32), tf.int32) - positive_count
negative_indices      = tf.random_shuffle(negative_indices)[:negative_count]

## 5.   Gather selected positive and negative ROIs
positive_rois         = tf.gather(proposals, positive_indices)
negative_rois         = tf.gather(proposals, negative_indices)
print(positive_rois.eval())

## 6.   Assign positive ROIs to GT boxes.
#      roi_gt_box_assignment shows for each positive overlap, which class has the maximum overlap
positive_overlaps     = tf.gather(overlaps, positive_indices)
roi_gt_box_assignment = tf.argmax(positive_overlaps, axis=1)
roi_gt_boxes          = tf.gather(gt_boxes    , roi_gt_box_assignment)
roi_gt_class_ids      = tf.gather(gt_class_ids, roi_gt_box_assignment)

print('     shape of positive overlaps is :', positive_overlaps.get_shape())
print(roi_gt_box_assignment.eval())
print(positive_overlaps.eval())

## 7.   Compute bbox delta 
# calculate refinement (difference b/w positive rois and gt_boxes) for positive ROIs
roi_gt_deltas  = utils.box_refinement_graph(positive_rois, roi_gt_boxes)
roi_gt_deltas /= config.BBOX_STD_DEV

## 8.  prepare gt_masks 
#      transpose gt_masks from [h, w, N] to [N, height, width] and add 4th dim at end [N, height, width, 1]
#      Pick the right mask for each ROI
# transposed_masks = tf.expand_dims(tf.transpose(gt_masks, [2, 0, 1]), -1)
# roi_masks = tf.gather(transposed_masks, roi_gt_box_assignment)

# Compute mask targets
# boxes = positive_rois

# if config.USE_MINI_MASK:
    # Transform ROI corrdinates from normalized image space
    # to normalized mini-mask space.
    # y1, x1, y2, x2 = tf.split(positive_rois, 4, axis=1)
    # gt_y1, gt_x1, gt_y2, gt_x2 = tf.split(roi_gt_boxes, 4, axis=1)
    # gt_h = gt_y2 - gt_y1
    # gt_w = gt_x2 - gt_x1
    # y1 = (y1 - gt_y1) / gt_h
    # x1 = (x1 - gt_x1) / gt_w
    # y2 = (y2 - gt_y1) / gt_h
    # x2 = (x2 - gt_x1) / gt_w
    # boxes = tf.concat([y1, x1, y2, x2], 1)

# box_ids = tf.range(0, tf.shape(roi_masks)[0])
# masks   = tf.image.crop_and_resize(tf.cast(roi_masks, tf.float32), 
                                   # boxes,
                                   # box_ids,
                                   # config.MASK_SHAPE)

# Remove the extra dimension from masks.
# masks = tf.squeeze(masks, axis=3)

# Threshold mask pixels at 0.5 to have GT masks be 0 or 1 to use with
# binary cross entropy loss.
# masks = tf.round(masks)

# Append negative ROIs and pad bbox roi_gt_deltas and masks that
# are not used for negative ROIs with zeros.
rois             = tf.concat([positive_rois, negative_rois], axis=0)
N                = tf.shape(negative_rois)[0]
P                = tf.maximum(config.TRAIN_ROIS_PER_IMAGE - tf.shape(rois)[0], 0)
rois             = tf.pad(rois            , [(0, P ), (0, 0)])

roi_gt_boxes     = tf.pad(roi_gt_boxes    , [(0, N + P), (0, 0)])
roi_gt_class_ids = tf.pad(roi_gt_class_ids, [(0, N + P)])
roi_gt_deltas    = tf.pad(roi_gt_deltas   , [(0, N + P), (0, 0)])
# masks            = tf.pad(masks           , [[0, N + P], (0, 0), (0, 0)])

# print(' roi_gt_boxes :  ' , tf.shape(roi_gt_boxes) )
# print(' P:  ' , P,  ' N :    ', N)   
# print('     roi.shape             :',  rois.shape            , tf.shape(rois))
# print('     roi_gt_class_ids.shape:',  roi_gt_class_ids.shape, tf.shape(roi_gt_class_ids))
# print('     roi_gt_deltas.shape   :',  roi_gt_deltas.shape   , tf.shape(roi_gt_deltas))
# print('     masks.shape           :',  masks.shape           , tf.shape(masks))
# print('     roi_gt_boxes.shape    :',  roi_gt_boxes.shape    , tf.shape(roi_gt_boxes))

#     return rois, roi_gt_class_ids,  roi_gt_deltas, roi_gt_boxes
#     return positive_ind_shuffled, positive_indices, positive_overlaps, roi_gt_box_assignment, roi_gt_boxes, roi_gt_class_ids

In [ ]:
print(tf.shape(rois).eval())
print(rois.eval())
print(roi_gt_class_ids.eval())
print(gt_class_ids.eval())
print(gt_boxes.eval())

In [ ]:
# sess = tf.InteractiveSession()
sess = KB.get_session()
# print(input_gt_boxes[0].eval())
# print(input_normlzd_gt_boxes[0].eval())
# print(norm_output_rois[0].eval(session=sess))


##  `build_pred_tf ()` 

###  Build Pred_Tensor

`pred_tensor[:,:,:,1:7]`  == `[116.9736  21.8213  36.2715  45.6026   0.    0.9139   ]`


Detections returned by `detect()` routine:

`[[ 25.          18.          80.          72.           2.           0.99936014]
  [ 51.           3.         106.          71.           3.           0.99924326]`

###  Setup tensors to be passed to `build_pred_tf ()`    

In [ ]:
# try:
#     sess.close()
#     print('session was deleted ')
# except:
#     print('Session was not defined ')
#     pass
# sess = tf.InteractiveSession()
mrcnn_class = tf.identity(mrcnn_class)
norm_output_rois = tf.identity(output_rois / [128,128,128,128])
target_class_ids = tf.identity(target_class_ids)
# in_tensor = tf.placeholder(tf.float32, shape=[3,4,32,6], name = 'in_tensor')
config = model.config
names = ['Dev']
print(mrcnn_class.shape)
print('output_rois')
print(norm_output_rois.shape)
print(norm_output_rois[:,:10].eval())
print('target_class_ids')
print(target_class_ids.eval())
print('input_gt_class_ids')
print(input_gt_class_ids)
print('input_gt_bboxes')
print(input_gt_bboxes[:,:10])
print('roi_gt_boxes')
print(roi_gt_boxes.shape)
print(roi_gt_boxes[:,:10])

### Original `build_prediction_tf()` for comparison

In [ ]:
def build_predictions_tf(mrcnn_class, norm_output_rois, config):
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES
    h, w            = config.IMAGE_SHAPE[:2]
    # num_rois        = config.TRAIN_ROIS_PER_IMAGE
    num_cols        = 6
    num_rois        = KB.int_shape(norm_output_rois)[1]
    output_rois = norm_output_rois * np.array([h,w,h,w])   

    print()
    print('  > BUILD_PREDICTIONS_TF()')
    print('    num_rois          : ', num_rois )
    print('    mrcnn_class shape : ', KB.shape(mrcnn_class), KB.int_shape(mrcnn_class))
    print('    mrcnn_bbox.shape  : ', KB.shape(mrcnn_bbox) , KB.int_shape(mrcnn_bbox), mrcnn_bbox.shape )
    print('    output_rois.shape : ', KB.shape(output_rois), KB.int_shape(output_rois))
    
    #---------------------------------------------------------------------------
    # use the argmaxof each row to determine the dominating (predicted) class
    #---------------------------------------------------------------------------
    pred_classes     = tf.to_int32(tf.argmax( mrcnn_class,axis=-1))
    pred_classes_exp = tf.to_float(tf.expand_dims(pred_classes ,axis=-1))
    pred_scores      = tf.reduce_max(mrcnn_class ,axis=-1, keepdims=True)   # (32,)
    
    # np.set_printoptions(linewidth=100, precision=4)
    # print('    pred_classes with highest scores:', pred_classes.get_shape() )
    # print('    pred_ scores:', pred_scores.get_shape())
    
    print('    pred_classes     : ', pred_classes.shape)
    print('    pred_classes_exp : ', pred_classes_exp.shape)
    print('    pred_scores      : ', pred_scores.shape)

    #-----------------------------------------------------------------------------------
    # This part is used if we want to gather bbox coordinates from mrcnn_bbox 
    #  Currently we are gathering bbox coordinates form output_roi so we dont need this
    #-----------------------------------------------------------------------------------
    # bbox_selected    = tf.zeros_like(norm_output_rois)
    # print('    bbox_selected    : ', bbox_selected.shape)
    # gather_boxes    = tf.stack([batch_grid, roi_grid, pred_classes, ], axis = -1)
    # print('-- gather_boxes  ----')
    # print('gather_boxes inds', type(gather_boxes), 'shape',tf.shape(gather_boxes).eval())
    # print(gather_boxes.eval())
    # bbox_selected   = tf.gather_nd(mrcnn_bbox, gather_boxes)
    # print('    bbox_selected shape : ', bbox_selected.get_shape())
    # print(bbox_selected[0].eval())    
    #-----------------------------------------------------------------------------------
    
    ## moved pred_scores to end -- 30-04-2018
    # pred_array  = tf.concat([bbox_idx , output_rois, pred_classes_exp , pred_scores], axis=-1)
    pred_array  = tf.concat([ output_rois, pred_classes_exp , pred_scores], axis=-1)

    #---------------------------------------------------------------------------
    # Build a meshgrid for image id and bbox to use in gathering of bbox information 
    #---------------------------------------------------------------------------
    batch_grid, roi_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32),
                                        tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )
    # bbox_idx             = tf.to_float(tf.expand_dims(roi_grid , axis = -1))    
    print('    batch_grid       : ', batch_grid.shape)
    print('    roi_grid         : ', roi_grid.shape)


    scatter_ind          = tf.stack([batch_grid , pred_classes, roi_grid],axis = -1)
    pred_scatt = tf.scatter_nd(scatter_ind, pred_array, [batch_size, num_classes, num_rois, pred_array.shape[-1]])
    # print('scatter_ind', type(scatter_ind), 'shape',tf.shape(scatter_ind).eval())
    # print('    pred_scatter shape is ', pred_scatt.get_shape(), pred_scatt)

    
    #------------------------------------------------------------------------------------
    ## sort pred_scatter in each class dimension based on sequence number (last column)
    #------------------------------------------------------------------------------------
    _, sort_inds = tf.nn.top_k(pred_scatt[:,:,:,-1], k=pred_scatt.shape[2])

    # build indexes to gather rows from pred_scatter based on sort order    
    class_grid, batch_grid, roi_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_rois))
    roi_grid_exp = tf.to_float(tf.expand_dims(roi_grid, axis = -1))
    
    gather_inds  = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
    pred_tensor  = tf.gather_nd(pred_scatt, gather_inds, name = 'pred_tensor')
    
    # print('    class_grid  ', type(class_grid) , 'shape', class_grid.get_shape())
    # print('    batch_grid  ', type(batch_grid) , 'shape', batch_grid.get_shape())
    # print('    roi_grid shape', roi_grid.get_shape(), 'roi_grid_exp shape ', roi_grid_exp.get_shape())
    # print('    gather_inds ', type(gather_inds), 'shape', gather_inds.get_shape())
    # print('    pred_tensor (gathered)  : ', pred_tensor.get_shape())

    # append an index to the end of each row --- commented out 30-04-2018
    # pred_tensor  = tf.concat([pred_tensor, roi_grid_exp], axis = -1)
    
    # print('    final pred_tensor shape  : ', pred_tensor.get_shape())
   # print('    complete')
    return  pred_tensor

### Copy of `build_prediction_tf()` for modification


###  Display for visual check - `pred_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('pred_tensor shape is ', pred_tensor.get_shape() )
img = 0
print('Image ', img , '/ Class 0 ------------')
print(pred_tensor[img,0].eval(session=sess))
print('Image ', img , '/ Class 1 ------------')
print(pred_tensor[img,1].eval(session=sess))
print('Image ', img , '/ Class 2 ------------')
print(pred_tensor[img,2].eval(session=sess))
print('Image ', img , '/ Class 3 ------------')
print(pred_tensor[img,3].eval(session=sess))


In [ ]:
 pred_tensor1  = build_predictions_tf(mrcnn_class, mrcnn_bbox, output_rois, model.config)

In [ ]:
img = 2
print(pred_tensor1[img,0].eval(session=sess))
print(pred_tensor1[img,1].eval(session=sess))
print(pred_tensor1[img,2].eval(session=sess))
print(pred_tensor1[img,3].eval(session=sess))

###  Test `means`, `covar`, `gauss_grid`, and `gauss_sum ` between development version and final version

In [ ]:
print(pred_tensor.get_shape(), pred_tensor2.get_shape())
print(np.all(pred_tensor.eval() == pred_tensor2.eval()))
# tst1 = means.eval()
# tst2 = means2.eval()
# print(tst1.shape, tst2.shape)

# print(tst1[0,:10])
# print()
# print(tst2[0,:10])
# print(np.all(tst1 == tst2))
# print()
# del tst1, tst2

In [ ]:
tst1 = pred_tensor.eval()
tst2 = pred_tensor2.eval()

In [ ]:
print(tst1[0])

In [ ]:
print(output_rois.eval())

## Experimental Code